# Make a simple connection and execute a SQL statement

In [7]:
import sqlite3
from sqlite3 import Error

In [8]:
try:
    conn = sqlite3.connect('data\PeopleAndPets.db')
except Error as e:
    print(e)

In [23]:
cur = conn.cursor()
cur.execute("SELECT * FROM people")
rows = cur.fetchall()
for row in rows:
    print(row)
cur.close()

(1, 'John', 'Funk')
(2, 'David', 'Brown')
(3, 'Megan', 'Macdonnald')
(4, 'Shane', 'Cornell')
(5, 'Shane', 'Cornell')
(6, 'Dorothy', 'Martinez')
(7, 'Dorothy', 'Martinez')
(8, 'Shaggy', 'Smith')


In [24]:
cur = conn.cursor()
cur.execute("SELECT * FROM pets")
rows = cur.fetchall()
for row in rows:
    print(row)
cur.close()

(1, 'Mazie', 1)
(2, 'Potatoe', 2)
(3, 'Tomatoe', 2)
(4, 'fluffy', '')
(5, 'Mr. Wiggles', 4)
(6, 'Mr. Wiggles', 5)
(7, 'Kringle', 9)
(8, 'Kringle', 9)
(9, 'Scooby', 8)


In [15]:
cur = conn.cursor()
cur.execute("select people.first_name,people.last_name, count(pets.pet_name) from people,pets where people.person_id = pets.owner_id group by pets.owner_id ;")
rows = cur.fetchall()
for row in rows:
    print(row)
cur.close()

('John', 'Funk', 1)
('David', 'Brown', 2)
('Shane', 'Cornell', 1)
('Shane', 'Cornell', 1)


# so what did fetchall() return?

In [12]:
row

('David', 'Brown', 2)

In [13]:
type(row)

tuple

In [19]:
row[0]

'David'

In [20]:
row[0:2]

('David', 'Brown')

# we really want a pandas dataframe

In [22]:
import pandas as pd

In [25]:
command="select people.first_name,people.last_name, count(pets.pet_name) from people,pets where people.person_id = pets.owner_id group by pets.owner_id ;"
df = pd.read_sql_query(command,conn)

In [26]:
df

,first_name,last_name,count(pets.pet_name)
0,John,Funk,1
1,David,Brown,2


# ok back to inserting rows with Python

### first we do a insert with a hard coded person id

In [9]:
person_first_name='Dorothy'
person_last_name='Martinez'
pet_name='Kringle'

person_sql_string = "insert into people (first_name, last_name) values ( '{0}','{1}'); ".format(person_first_name,person_last_name)
person_id=9 #Pick a number that works with the real database
pet_sql_string = "insert into pets (pet_name, owner_id) values ( '{0}',{1} ) ;".format(pet_name, person_id)

In [10]:
print(person_sql_string)
print(pet_sql_string)

insert into people (first_name, last_name) values ( 'Dorothy','Martinez'); 
insert into pets (pet_name, owner_id) values ( 'Kringle',9 ) ;


In [13]:
cur = conn.cursor()
cur.execute(person_sql_string)
cur.execute(pet_sql_string)
cur.close()

### next we'll make it more realistic and do both inserts in the scope of a transaction

In [18]:
person_first_name='Shaggy'
person_last_name='Smith'
pet_name='Scooby'

person_sql_string = "insert into people (first_name, last_name) values ( '{0}','{1}'); ".format(person_first_name,person_last_name)
pet_sql_string = "insert into pets (pet_name, owner_id) values ( '{0}',last_insert_rowid() ) ;".format(pet_name)
cur = conn.cursor()  #automaticly starts a transaction
cur.execute(person_sql_string)
cur.execute(pet_sql_string)
cur.close() # automaticly commits the transaction